# Stomata detection & recording program

In [ ]:
#requirements
#opencv, tensorflow
#clone tensorflow models into the same directory as this ipynb notebook
#ex dowonload through colab
#!git clone https://github.com/tensorflow/models.git

### How to run
Activate the cell below by clicking somewhere in the cell, then run by clicking [▶︎Run] (or pressing Shift+Enter key). Wait for "ln [ * ]" leftside of the cell to change into "ln [1]" or other number in [ ]. At the same time, top right environment(virtualenv_base)[○] sign will change into [●]. It may take a minute.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import cv2
import tensorflow as tf
import time
import numpy as np
import matplotlib.pyplot as plt
import os, sys
%matplotlib inline
import pme

sys.path.append("models/research/")
sys.path.append("models/research/slim")
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from IPython.display import Image, display, clear_output

from google.protobuf import text_format



category_index = label_map_util.create_category_index_from_labelmap("labelmap.pbtxt",
                                                                    use_display_name=True)

PATH_TO_SAVED_MODEL = "saved_model"
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

def draw_bboxes_and_return_meta(
        image,
        detections,
        line_thickness=2,
        min_score_thresh=.3,
        COLOR='LimeGreen'):
    global dets
    dets = [[x, z] for x, y, z in zip(detections["detection_boxes"], detections["detection_classes"],
                                      detections["detection_scores"]) if y == 1 and z > min_score_thresh]
    if len(dets)!=0:
        dets.sort(key=lambda r: r[0][0])
        dets = [[i, *x] for i, x in enumerate(dets)]
        dets = np.array(dets)
        height, width = image.shape[0], image.shape[1]
        dets[:,1] = [[a*height,b*width,c*height,d*width] for a,b,c,d in dets[:,1]]   

        for det in dets:
            box = det[1]
            ymin, xmin, ymax, xmax = box

            x = (xmin+xmax)//2
            y = (ymin+ymax)//2
            string = '{}:{:.2g}% '.format(det[0],round(100*det[2]))

            viz_utils.draw_bounding_box_on_image_array(
                image,
                ymin,
                xmin,
                ymax,
                xmax,
                color=COLOR,
                thickness=line_thickness,
                display_str_list=[string],
                use_normalized_coordinates=False)

    return image, dets

def pipeline_func(image):
    image_np = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = image_np.copy()
    image_np = cv2.cvtColor(image_np,cv2.COLOR_GRAY2RGB)
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]
    detections = detect_fn(input_tensor)
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
    image_np, dets = draw_bboxes_and_return_meta(
        image_np,
        detections,
        line_thickness = 4,
        min_score_thresh=.40)    
    
    nobj = 0 if len(dets) ==0 else len(dets[:,0])

    s = []
    
    for det in dets:
        idx,box,score = det
        ymin, xmin, ymax, xmax = box
        x = (xmin+xmax)//2
        y = (ymin+ymax)//2
        height = ymax - ymin
        width = xmax - xmin
        s.append([idx,score,y,x,height,width, *box])
    
    ret = {
        "images": [gray,cv2.cvtColor(image_np,cv2.COLOR_RGB2BGR)],
        "stream_logs": "number of stomata:" + str(nobj),
        "csv_logs": s,
        "csv_header": ["stomata_id","score", "ycoord", "xcoord", "height", "width","bbox_ymin","bbox_xmin","bbox_ymax","bbox_xmax"]        
    }
    return ret

## GUI program

Run the cell below to start GUI for monitoring and acquisition of the camera images. It may take 10 - 20 sec to  start.
You can run it even before cell1 above become completed. That case, cell below will be run after the cell above completed.

In [ ]:
_ = pme.stream(
    pipeline_func=pipeline_func,
    output_directory = None,
    camera_id=0, videocapture_api_backend=200,
    camera_initial_settings={'format': ['M', 'J', 'P', 'G'], 'height': 768, 'width': 1024, 'fps': 30})

- images will be saved to /home/phytometrics/Desktop/microscope/data
- csv_logs will be written to /home/phytometrics/Desktop/microscope/data/20200904_134537.csv


### The arguments for pme.stream
- pipeline_func : image processing function
- output_directory: the directory to save csv files and image files. If there is no argument or "None", saved  in "/home/phytometrics/Desktop/microscope/data" as default. 
- camera_id: camera ID number. if there is only one camera connected, 0 is default.
- videocapture_api_backend: v4l2 connection. DO NOT EDIT. For developer only!
- camera_initial_settings: camera settings for image format, size and frame rate. Edit only when you connect a new camera. DO NOT EDIT otherwise.

In [ ]:
#To know the directory name of the USB memory inserted, run this command.
!ls /media/phytometrics   #ex. XXXX-XXXX -> set output_directory as "/media/XXXX-XXXX/data" if you want to save the files in the USB memory.

6011-44C3


## contents in GUI window
- images will be saved to ..., csv_logs will be written to
    - A directory where the acquired images and the csv file are saved. The name of csv file is a timestamp when you run the cell.
- Connect
    - start connection to the usb camera.
- Pause
    - stop the input from the camera tentatively. When you switch from “Connect” to “Pause”, camera will be still active, so can quickly become back to “Connect”. However, if you would like to do other tasks for a while, “disconnect” is recommended because “Pause” continues  to make a certain load onto the computer.
- disconnect
    - Disconnect the camera. It can become back to “Connect” and save data into the same directory and csv file.
- exit
    - Finish the program. After this, you can start another round of observation by running the cell again.
- acquire image
    - Save the current image. If it is “Pause” mode, the image being displayed will be saved. When “enable image analysis pipeline” (see below) is checked, the grayscale image (analysis input) and stomata-marked image (analysis output) are also saved, and stomatal data are written into the csv.
- Enable image analysis pipeline
    - Apply the stomata detection program. When it is checked, the grayscale image (analysis input) and stomata-marked image (analysis output) are displayed.
- Display Stream Logs
    - Display the detected stomatal number.
- Hide Input
    - Switch ON/OFF of camera input which is an RGB color image. Regardless of checked or not, the camera input image will be saved after you press “acquire image” button.
    
## precautions
- The application may freeze by trying reconnection when the camera connection is already active. If you would like to rerun the cell, do it after inactivate camera connection by press “disconnect” or “exit”.
- In the case the application has become frozen, click the interrupt button [■] (upper part in the window), choose Restart from Kernel drop-down list (just above [■]), and reload the page by the browser’s command. These procedures will refresh the page.

### To analyze images you already have, do below.

In [ ]:
import csv

DIRECTORY = "samples" #saving directory
files = os.listdir(DIRECTORY)
print(files) #the images to be analyzed

In [ ]:
##analyze only a image file

file = files[0]
path = os.path.join(DIRECTORY,file) #image path
image = cv2.imread(path)  #load the image
ret = pipeline_func(image)  #analysis by the network
##pipeline_func outputs:
##dictionary. key consists of images, stream_logs, csv_logs, csv_header. Only images and csv_logs are processed.

##To display the image, remove hash symbols below and run.
#gray, annotated = ret["images"]
#plt.imshow(gray,cmap="gray")
#plt.show()
#plt.imshow(annotated)
#plt.show()

##To save the csv file, remove hash symbols below and run.
#name_of_csv = "test.csv"
#header = ["stomata_id","score", "ycoord", "xcoord", "height", "width","bbox_ymin","bbox_xmin","bbox_ymax","bbox_xmax"] 
#with open(name_of_csv, "a") as f: #adding mode
#        writer = csv.writer(f, lineterminator='\n')
#        writer.writerow(header) #write header
#        for stomata in ret["csv_logs"]: #stomatal data
#                s = [file, *stomata] #image file name and stomatal data
#                writer.writerow(s)

##To save the image, remove hash symbols below and run.
#cv2.imsave("test.jpg",gray)
#cv2.imsave("test2.jpg",gray)

In [ ]:
##analyze all files in a directory

##csv header is located out of the loop for "file"
#name_of_csv = "test.csv"
#header = ["stomata_id","score", "ycoord", "xcoord", "height", "width","bbox_ymin","bbox_xmin","bbox_ymax","bbox_xmax"] 
#with open(name_of_csv, "a") as f: #adding mode
#        writer = csv.writer(f, lineterminator='\n')
#        writer.writerow(header) #write header

for file in files:
    print(file, end=" ")
    path = os.path.join(DIRECTORY,file)
    image = cv2.imread(path) 
    ret = pipeline_func(image) 
    
    ##below is to save csv
    #with open(name_of_csv, "a") as f:  
    #        writer = csv.writer(f, lineterminator='\n')
    #        for stomata in ret["csv_logs"]:
    #                s = [file, *stomata] 
    #                writer.writerow(s)
    
    ##To save the image, run below. Here is a case the image file is xxxxx.jpg
    #base = os.path.splitext(file)　#get the file name xxxxx
    #gray_file_name = base +"_gray.jpg" #additional characters for grayscale image name. 
    ##you can save the image by cv2.imsave function

In [ ]:
## In case the resolution is so high that the detection is not effective,
## adjust the image size before analysis. You can resize as below or with other software.

## define the absolute size
#size = (768,1024)
#image = cv2.resize(image, size)

##define the size ratio. Below is 1/2.
#image = cv2.resize(image,None, fx=1/2,fy=1/2)

##then, you can analyze it like “ret = pipline_func(image)”

## Terminal commands to download the files and directories from the Jetson nano


directory download:

$ scp jetson@192.168.55.1:/home/phytometrics/Desktop/microscope/data/XXXX.jpg "a directory in your computer"/data

file download:

$ scp jetson@192.168.55.1:/home/phytometrics/Desktop/microscope/data/XXXX.jpg "a directory in your computer"/xxxx.jpg

You will be requested for a password for the Jetson nano.